In [1]:
import pandas as pd

In [6]:
train = pd.read_csv("diamonds_train.csv")
train = train.drop(train.columns[0], axis=1)
test = pd.read_csv("diamonds_test.csv")
test = test.drop(test.columns[0], axis=1)